In [1]:
import sqlite3
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain.llms import Ollama
from langchain_core.prompts import PromptTemplate

### Step 1: Convert CSV to SQLite Database

In [2]:
csv_file = "data/diabetes.csv"  # Update with your actual CSV file name
db_file = "database.db"  # SQLite database name
table_name = "diabetes"

In [3]:
df = pd.read_csv(csv_file)

# Connect to SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
df.to_sql(table_name, conn, if_exists="replace", index=False)

768

In [4]:
# Fetch schema from SQLite
cursor.execute(f"PRAGMA table_info({table_name})")
schema_info = cursor.fetchall()

# Generate schema string
schema_str = "\n".join([f"{col[1]} ({col[2]})" for col in schema_info])
conn.commit()
conn.close()

print(f"Database Schema:\n{schema_str}")

Database Schema:
Pregnancies (INTEGER)
Glucose (INTEGER)
BloodPressure (INTEGER)
SkinThickness (INTEGER)
Insulin (INTEGER)
BMI (REAL)
DiabetesPedigreeFunction (REAL)
Age (INTEGER)
Outcome (INTEGER)


### Step 2: Initialize LLaMA 3 (via Ollama API)

In [5]:
llm = Ollama(model="llama3")

# Define a prompt for LLaMA to generate SQL queries
prompt_template = PromptTemplate.from_template(
    """You are an AI assistant that generates SQL queries.
    The database contains a table named `{table_name}` with the following schema:

    {schema}

    Generate an SQL query to answer: "{question}"
    Use the exact table name `{table_name}` in the query.
    Only return the SQL query, no explanation.
    """
)

C:\Users\Aman\AppData\Local\Temp\ipykernel_25040\1234702271.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


In [6]:
def generate_sql_query(question):
    """Uses LLaMA 3 to generate an SQL query based on schema and user question."""
    prompt = prompt_template.format(schema=schema_str, table_name=table_name, question=question)
    response = llm.invoke(prompt)
    return response.strip()

### Step 3: Query Execution Function

In [7]:
def execute_query(sql_query):
    """Executes the generated SQL query on SQLite and returns the result."""
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        conn.close()
        return f"Error executing query: {e}"

### Step 4: Ask a question and get the answer

In [8]:
user_question = "What is the average age of diabetic patients?"
generated_query = generate_sql_query(user_question)
print("Generated SQL Query:", generated_query)

query_result = execute_query(generated_query)
print("Query Result:", query_result)

Generated SQL Query: SELECT AVG(Age) FROM diabetes WHERE Outcome = 1;
Query Result: [(37.06716417910448,)]
